In [100]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
import os
import glob
import pandas as pd
import time
import random
import codecs
import shutil
from datetime import datetime
import re

In [129]:
def scraping_to_dataframe():
    step = 1
    
    def extract_int(sample):
        list = re.findall(r'\d+', sample)
        while True:
            try:
                res = int("".join(map(str, list)))
                break
            except ValueError:
                res = np.nan
                break
        return res
    success = 0
    fail = 0
    try:
        source = 'as_24'

        # Start time
        start_time = datetime.now()

        # directory of html annonces
        directory = 'annonces'

        # log update
        log_import = pd.read_csv('log.csv')
        log_new = pd.DataFrame({'source': [source],
                                'step': ['to dataframe'],
                                'status': ['started'],
                                'time': [datetime.now()],
                                'details': [""]})
        log = log_import.append(log_new, ignore_index=True)
        log.to_csv('log.csv', index=False)

        # import index
        index_df = pd.read_csv('as_24_annonces_light.csv')
        #loop sur le html de chaque annonce
        
        #ouverture de annonces_light dans lequel figure l'url pour le guid
        annonces_light = pd.read_csv('as_24_annonces_light.csv')
        
        for filename in [file for file in os.listdir(directory) if file.endswith(".html")]:
            #initialize the DataFrame
            #============================================================================
            columns=['reference', 'model', 'price', 'mileage', 'power_kW', 'power_CH', 
                             'État', 'Propriétaires préc.', 'Dernier contrôle technique', 'Garantie', 
                             "Carnet d'entretien", 'Marque', "N° d'annonce", 'Année', 'Couleur extérieure', 
                            'Type de peinture', 'Couleur originale', 'Carrosserie', 'Transmission', 'Vitesses', 
                            'Cylindrée', 'Cylindres', 'Poids à vide', 'Description','url']
            
            df_announces = pd.DataFrame(columns=columns)
            
            #Différentes parties de l'annonce et du DataFrame
            part_one_row = {"reference": np.nan,
                  "model": "",
                  "price": np.nan,
                  "mileage": 0,
                  "power_kW": 0,
                  "power_CH": 0}
            etat_features = ["État", "Propriétaires préc.", "Dernier contrôle technique", "Garantie", "Carnet d'entretien"]
            etat_row = {"État": "", 
                      "Propriétaires préc.": np.nan,
                      "Dernier contrôle technique": datetime(1970, 1, 1),
                      "Garantie": datetime(1970, 1, 1),
                      "Carnet d'entretien": np.nan}
            car_features = ["Marque", "N° d'annonce", "Année", "Couleur extérieure", "Type de peinture", "Couleur originale", "Carrosserie"]
            car_row = {"Marque": "", "N° d'annonce": np.NaN, 
                      "Année": [""], 
                      "Couleur extérieure": "", 
                      "Type de peinture": "", 
                      "Couleur originale": "", 
                      "Carrosserie": ""}
            transm_features = ["Transmission", "Vitesses", "Cylindrée", "Cylindres", "Poids à vide"]
            transm_row = {"Transmission": "", 
                          "Vitesses": np.nan, 
                          "Cylindrée": np.nan, 
                          "Cylindres": 0, 
                          "Poids à vide" : np.nan,
                          "Description" : "",
                          "url": ""
                         }
            
            #request depuis les fichiers html
            f = codecs.open(f"{directory}/{filename}", 'r')
            soup_ = BeautifulSoup(f, "html.parser")
            reference = filename.split('-')[1] #soit le guid donc ref = guid
            uniq_id = source + '-' + reference
#             file_name = source + '-' + announces_row['guid'] + '-' + start_time.strftime("%Y-%m-%d_%Hh%M")

            try:
                #première encart informations majeures de l'annonce
                guid = soup_.find("div", class_="sc-pull-right cldt-buttons cldt-hide-on-print cldt-version-one")
                guid = guid.find("a", class_="btn-watchlist").attrs['data-classified-guid']
                part_one_row["reference"] = "as24_" + guid
                part_one_row["model"] = soup_.find("div", class_="cldt-headline").find("span", class_="cldt-detail-makemodel sc-ellipsis").text
                part_one_row["price"] = extract_int(soup_.find("div", class_="cldt-price").find('h2').text)

                basic_data = soup_.find("div", class_='cldt-stage-basic-data')
                part_one_row["mileage"] = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[0].string)
                part_one_row["power_kW"] = extract_int(basic_data.find_all("span", class_="sc-font-l cldt-stage-primary-keyfact")[2].string)
                part_one_row["power_CH"] = extract_int(basic_data.find_all("span", class_="sc-font-m cldt-stage-primary-keyfact")[0].string)

                #scrapping de la branche contenant les key/values de Etat
                etat = soup_.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")
                #création liste des key et des valeurs => création dictionnaire de key, values
                etat_key = [i.text for i in etat.find_all("dt")]
                etat_value = [i.text.replace('\n', '') for i in etat.find_all("dd")]
                etat_dict = {k: v for k,v in zip(etat_key, etat_value)}
                #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
                for k_site in [i.text for i in etat.find_all("dt")]:
                    for k_list in etat_features:
                        if k_site == k_list:
                            etat_row[k_list]=etat_dict[k_site]
                            
                #scrapping de la branche contenant les key/values de caractéristiques
                car = soup_.find("div", class_="cldt-categorized-data cldt-data-section sc-pull-right")
                #création liste des key et des valeurs => création dictionnaire de key, values
                car_key = [i.text for i in car.find_all("dt")]
                car_value = [i.text.replace('\n', '') for i in car.find_all("dd")]
                car_dict = {k: v for k,v in zip(car_key, car_value)}
                #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
                for k_site in [i.text for i in car.find_all("dt")]:
                    for k_list in car_features:
                        if k_site == k_list:
                            car_row[k_list]=car_dict[k_site]
                            
                #scrapping de la branche contenant les key/values de caractéristiques
                transm = soup_.find_all("div", class_="cldt-categorized-data cldt-data-section sc-pull-left")[1]
                #création liste des key et des valeurs => création dictionnaire de key, values
                transm_key = [i.text for i in transm.find_all("dt")]
                transm_value = [i.text.replace('\n', '') for i in transm.find_all("dd")]
                transm_dict = {k: v for k,v in zip(transm_key, transm_value)}
                #crée un dictionnaire correspondant à la ligne qu'il faudra ajouter au dataframe
                for k_site in [i.text for i in transm.find_all("dt")]:
                    for k_list in transm_features:
                        if k_site == k_list:
                            transm_row[k_list]=transm_dict[k_site]
                transm_row["Description"]=soup_.find_all("div", class_="sc-grid-col-6 sc-grid-col-s-12")[1].text
                transm_row["Description"]=transm_row["Description"].replace('\n','').replace('Afficher plus','').replace('Afficher moins','')
                
                #========= test de l'ajout de l'url dans le dataframe ============
                #guid existe déjà plus haut
                guid_full = part_one_row["reference"].replace('as24_','')
                #identification de la ligne du guid dans annonces_light &extract url de annonce light
                url = annonces_light.loc[annonces_light['guid']==guid_full].iloc[0,3]
                transm_row["url"]=url                
                #=================================================================
                
                
                #========= ajouter la sauvegarde d'images ======================
                
                container = soup_.find_all('div',class_="gallery-picture")
                images_list = []
                uniq_id = 'as24' + '-' + guid
                
                if reference not in [file.split('-')[1] for file in glob.glob('img/*')]:
                    for image in container:
                        images_list.append(image.find("img").attrs['data-fullscreen-src'])

                    k = 0
                    for image_url in images_list[0:3]:
                        img_data = requests.get(image_url).content
                        with open(f'images/{uniq_id}-{k}.jpg', 'wb') as handler:
                            handler.write(img_data)
                            image = Image.open(f'images/{uniq_id}-{k}.jpg') 
                            ratio = image.size[0] / image.size[1]
                            image = image.resize((300,int(300/ratio)))
                            image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 50)
                        k+=1
                #===============================================================
                
                #concaténation des différents dataframes
                csv_destination = 'as_24_annonces_full.csv'
                
                dall = {}
                dall.update(part_one_row)
                dall.update(etat_row)
                dall.update(car_row)
                dall.update(transm_row)
                df_dall = pd.DataFrame([dall])
                df_dall = df_dall.reindex(columns = columns)
                
                #dictionnaire pour renommer les colonnces
                rename_columns = {'État':'etat', 'Propriétaires préc.':'proprietaires_prec.', 
                                  'Dernier contrôle technique':'dernier_controle_technique', 'Garantie':'garantie', 
                                  "Carnet d'entretien":"carnet_dentretien", 'Marque':'marque', "N° d'annonce":"n_dannonce",
                                  'Année':'annee', 'Couleur extérieure':'couleur_exterieure',
                                  'Type de peinture':'type_de_peinture', 'Couleur originale':'couleur_originale',
                                  'Carrosserie':'carrosserie', 'Transmission':'transmission', 'Vitesses':'vitesses',
                                  'Cylindrée':'cylindree', 'Cylindres':'cylindres', 'Poids à vide':'poids_a_vide', 
                                  'Description':'description', 'url':'url'}
                
                #ici je renomme les colonnes du df
                df_dall = df_dall.rename(columns=rename_columns)
                
                # import history
                history = pd.read_csv(csv_destination)

                #concatenate new and history
                final_df = history.append(df_dall, ignore_index=True)
                
                # export to csv
                final_df.to_csv(csv_destination, index=False)
                print(step, ' : success scraping')
                success +=1
            except:
                print(step, ' : failed scraping')
                fail +=1
                
            # move file to vault after process
            # source path
            source_folder = f"annonces/{filename}"
            # destination path
            destination = f"annonces/vault/{filename}"
            # Move the content of
            # source to destination
            shutil.move(source_folder, destination)
            
#pas besoin sleep puisque scrap le fichier html en local
#             time.sleep(random.randint(1, 2))

            step +=1
        # End time
        end_time = datetime.now()
        td = end_time - start_time

        # log update
        log_import = pd.read_csv('log.csv')
        log_new = pd.DataFrame({'source': [source],
                                'step': ['to dataframe'],
                                'status': ['completed'],
                                'time': [datetime.now()],
                                'details': [f"{td.seconds/60} minutes elapsed"]})
        log = log_import.append(log_new, ignore_index=True)
        log.to_csv('log.csv', index=False)

    except (ValueError, TypeError, NameError, KeyError, RuntimeWarning) as err:
        print('failed moving file')
        # log update
        log_import = pd.read_csv('log.csv')
        log_new = pd.DataFrame({'source': [source],
                                'step': ['to dataframe'],
                                'status': ['error'],
                                'time': [datetime.now()],
                                'details': [err]})
        log = log_import.append(log_new, ignore_index=True)
        log.to_csv('log.csv', index=False)
    print('nombre de motos importées : ', success)
    print('nombre de motos non-importées : ', fail)
    print('ratio total : ', (success/(succes + fail)))
    print('over & out')

In [131]:
scraping_to_dataframe()

1  : success scraping
2  : success scraping
3  : success scraping
4  : success scraping
5  : success scraping
6  : success scraping
7  : success scraping
8  : failed scraping
9  : failed scraping
10  : success scraping
11  : success scraping
12  : success scraping
13  : success scraping
14  : success scraping
15  : success scraping
16  : success scraping
17  : success scraping
18  : success scraping
19  : success scraping
20  : failed scraping
21  : success scraping
22  : success scraping
23  : success scraping
24  : success scraping
25  : success scraping
26  : success scraping
27  : success scraping
28  : failed scraping
29  : success scraping
30  : success scraping
31  : success scraping
32  : success scraping
33  : success scraping
34  : success scraping
35  : success scraping
36  : success scraping
37  : success scraping
38  : success scraping
39  : success scraping
40  : success scraping
41  : success scraping
42  : success scraping
43  : success scraping
44  : failed scraping
45

350  : success scraping
351  : success scraping
352  : failed scraping
353  : success scraping
354  : success scraping
355  : success scraping
356  : success scraping
357  : success scraping
358  : success scraping
359  : failed scraping
360  : success scraping
361  : success scraping
362  : success scraping
363  : failed scraping
364  : failed scraping
365  : success scraping
366  : success scraping
367  : success scraping
368  : success scraping
369  : success scraping
370  : success scraping
371  : success scraping
372  : success scraping
373  : success scraping
374  : success scraping
375  : success scraping
376  : success scraping
377  : success scraping
378  : failed scraping
379  : success scraping
380  : success scraping
381  : success scraping
382  : success scraping
383  : success scraping
384  : success scraping
385  : success scraping
386  : success scraping
387  : success scraping
388  : success scraping
389  : success scraping
390  : success scraping
391  : success scrapi

694  : failed scraping
695  : success scraping
696  : success scraping
697  : success scraping
698  : success scraping
699  : success scraping
700  : success scraping
701  : success scraping
702  : success scraping
703  : success scraping
704  : success scraping
705  : success scraping
706  : success scraping
707  : success scraping
708  : success scraping
709  : success scraping
710  : success scraping
711  : success scraping
712  : success scraping
713  : success scraping
714  : success scraping
715  : success scraping
716  : success scraping
717  : success scraping
718  : success scraping
719  : failed scraping
720  : success scraping
721  : success scraping
722  : success scraping
723  : success scraping
724  : success scraping
725  : success scraping
726  : success scraping
727  : success scraping
728  : success scraping
729  : success scraping
730  : success scraping
731  : success scraping
732  : success scraping
733  : success scraping
734  : success scraping
735  : success scr

1036  : success scraping
1037  : success scraping
1038  : success scraping
1039  : success scraping
1040  : success scraping
1041  : success scraping
1042  : success scraping
1043  : success scraping
1044  : success scraping
1045  : success scraping
1046  : success scraping
1047  : success scraping
1048  : success scraping
1049  : success scraping
1050  : success scraping
1051  : success scraping
1052  : success scraping
1053  : failed scraping
1054  : failed scraping
1055  : success scraping
1056  : success scraping
1057  : success scraping
1058  : success scraping
1059  : success scraping
1060  : success scraping
1061  : success scraping
1062  : success scraping
1063  : success scraping
1064  : success scraping
1065  : success scraping
1066  : success scraping
1067  : success scraping
1068  : success scraping
1069  : failed scraping
1070  : success scraping
1071  : success scraping
1072  : success scraping
1073  : success scraping
1074  : success scraping
1075  : failed scraping
1076

1366  : success scraping
1367  : success scraping
1368  : success scraping
1369  : success scraping
1370  : success scraping
1371  : success scraping
1372  : success scraping
1373  : success scraping
1374  : success scraping
1375  : success scraping
1376  : failed scraping
1377  : failed scraping
1378  : success scraping
1379  : success scraping
1380  : success scraping
1381  : success scraping
1382  : failed scraping
1383  : success scraping
1384  : success scraping
1385  : success scraping
1386  : success scraping
1387  : failed scraping
1388  : success scraping
1389  : success scraping
1390  : success scraping
1391  : failed scraping
1392  : success scraping
1393  : success scraping
1394  : success scraping
1395  : success scraping
1396  : success scraping
1397  : success scraping
1398  : success scraping
1399  : failed scraping
1400  : success scraping
1401  : success scraping
1402  : success scraping
1403  : failed scraping
1404  : success scraping
1405  : success scraping
1406  :

...

2356  : failed scraping
2357  : success scraping
2358  : success scraping
2359  : success scraping
2360  : success scraping
2361  : failed scraping
2362  : success scraping
2363  : success scraping
2364  : success scraping
2365  : success scraping
2366  : success scraping
2367  : success scraping
2368  : success scraping
2369  : success scraping
2370  : success scraping
2371  : success scraping
2372  : success scraping
2373  : success scraping
2374  : success scraping
2375  : success scraping
2376  : success scraping
2377  : failed scraping
2378  : success scraping
2379  : success scraping
2380  : success scraping
2381  : success scraping
2382  : success scraping
2383  : failed scraping
2384  : success scraping
2385  : success scraping
2386  : success scraping
2387  : failed scraping
2388  : success scraping
2389  : success scraping
2390  : success scraping
2391  : success scraping
2392  : success scraping
2393  : success scraping
2394  : success scraping
2395  : success scraping
2396 

2686  : failed scraping
2687  : success scraping
2688  : success scraping
2689  : success scraping
2690  : success scraping
2691  : success scraping
2692  : failed scraping
2693  : success scraping
2694  : success scraping
2695  : success scraping
2696  : success scraping
2697  : failed scraping
2698  : success scraping
2699  : success scraping
2700  : success scraping
2701  : success scraping
2702  : success scraping
2703  : success scraping
2704  : failed scraping
2705  : success scraping
2706  : success scraping
2707  : failed scraping
2708  : success scraping
2709  : success scraping
2710  : success scraping
2711  : success scraping
2712  : success scraping
2713  : success scraping
2714  : success scraping
2715  : success scraping
2716  : success scraping
2717  : success scraping
2718  : success scraping
2719  : success scraping
2720  : success scraping
2721  : success scraping
2722  : success scraping
2723  : success scraping
2724  : success scraping
2725  : success scraping
2726 

3016  : failed scraping
3017  : failed scraping
3018  : success scraping
3019  : success scraping
3020  : success scraping
3021  : success scraping
3022  : success scraping
3023  : success scraping
3024  : success scraping
3025  : success scraping
3026  : success scraping
3027  : success scraping
3028  : success scraping
3029  : success scraping
3030  : success scraping
3031  : success scraping
3032  : success scraping
3033  : success scraping
3034  : success scraping
3035  : success scraping
3036  : success scraping
3037  : success scraping
3038  : success scraping
3039  : success scraping
3040  : success scraping
3041  : success scraping
3042  : success scraping
3043  : success scraping
3044  : failed scraping
3045  : success scraping
3046  : failed scraping
3047  : success scraping
3048  : success scraping
3049  : success scraping
3050  : failed scraping
3051  : success scraping
3052  : success scraping
3053  : failed scraping
3054  : success scraping
3055  : success scraping
3056  

3346  : success scraping
3347  : success scraping
3348  : success scraping
3349  : success scraping
3350  : success scraping
3351  : success scraping
3352  : success scraping
3353  : success scraping
3354  : success scraping
3355  : success scraping
3356  : success scraping
3357  : success scraping
3358  : success scraping
3359  : success scraping
3360  : success scraping
3361  : success scraping
3362  : success scraping
3363  : success scraping
3364  : failed scraping
3365  : success scraping
3366  : failed scraping
3367  : success scraping
3368  : success scraping
3369  : failed scraping
3370  : success scraping
3371  : success scraping
3372  : success scraping
3373  : success scraping
3374  : success scraping
3375  : success scraping
3376  : failed scraping
3377  : success scraping
3378  : success scraping
3379  : success scraping
3380  : failed scraping
3381  : success scraping
3382  : success scraping
3383  : success scraping
3384  : success scraping
3385  : success scraping
3386 

3676  : success scraping
3677  : success scraping
3678  : success scraping
3679  : success scraping
3680  : success scraping
3681  : success scraping
3682  : success scraping
3683  : success scraping
3684  : success scraping
3685  : success scraping
3686  : failed scraping
3687  : success scraping
3688  : success scraping
3689  : success scraping
3690  : success scraping
3691  : success scraping
3692  : success scraping
3693  : success scraping
3694  : success scraping
3695  : success scraping
3696  : success scraping
3697  : success scraping
3698  : success scraping
3699  : success scraping
3700  : success scraping
3701  : success scraping
3702  : success scraping
3703  : success scraping
3704  : success scraping
3705  : success scraping
3706  : success scraping
3707  : success scraping
3708  : failed scraping
3709  : success scraping
3710  : success scraping
3711  : success scraping
3712  : success scraping
3713  : success scraping
3714  : success scraping
3715  : success scraping
37

In [132]:
pd.read_csv('as_24_annonces_full.csv')

,reference,model,price,mileage,power_kW,power_CH,etat,proprietaires_prec.,dernier_controle_technique,garantie,...,couleur_exterieure,type_de_peinture,couleur_originale,carrosserie,transmission,vitesses,cylindree,cylindres,poids_a_vide,description
0,as24_0a9d3930-ed99-47a5-a9aa-e1de84a96101,Triumph Thruxton,6490,23090.0,NaN,NaN,Occasion,NaN,1970-01-01,1970-01-01,...,Bleu,NaN,BLEU,Roadster,NaN,NaN,900 cm³,0,NaN,Options : Garantie 3 mois avec possibilit d'ex...
1,as24_00a43cb8-9780-de17-e053-0100007f2913,Suzuki,7500,10.0,26.0,35.0,Occasion,1.0,1970-01-01 00:00:00,1970-01-01 00:00:00,...,Jaune,NaN,NaN,NaN,NaN,5.0,400 cm³,1,98 kg,"vends Suzuki TM 400 1974Vends Moto de cross, S..."
2,as24_00e21fc4-b5f3-9150-e053-0100007f070d,Honda VF 1100,4000,15500.0,92.0,125.0,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,Noir,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,"Vends Honda V65 Sabre de 1984, modèle VF1100S ..."
3,as24_0a54b3e8-dad4-4b1a-93d5-e875389f9dd3,Mash,3499,302.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,NaN,NaN,NaN,Roadster,NaN,NaN,400 cm³,0,NaN,ETAT NEUF.. !!! ( vu les kms ) une vidange ann...
4,as24_00e5dc18-9e54-4375-929f-6881bf8b09db,Husqvarna,4999,2016.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,NaN,NaN,NaN,Cross,NaN,NaN,125 cm³,0,NaN,?CBO MOTO AGEN Concessionnaire HONDA / HUSQVAR...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291,as24_abe130e2-de74-4ee2-b831-6f07876f0592,Peugeot Pulsion,3790,3870.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,Bleu,NaN,BLEU,Scooter,NaN,NaN,125 cm³,0,NaN,Options : Garantie 3 mois avec possibilit d'ex...
3292,as24_60692618-32e4-422e-b7e4-3190d5ee112f,Honda SH 125,2099,15900.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,NaN,NaN,NaN,Scooter,NaN,NaN,125 cm³,0,NaN,AUVERGNE MOTOS VEND UN SH 125 DU 25/01/2013 OP...
3293,as24_20949524-9153-4805-aec8-865096635deb,Yamaha XSR 900,7990,16900.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,Rouge,NaN,Rouge,Roadster,NaN,NaN,850 cm³,0,NaN,LIGNE SG PROJECT / SUPPORT DE PLAQUE / CLIGNO ...
3294,as24_63c4af38-45c3-4fa0-b729-9a78980005a2,KTM 790 Duke,7890,26911.0,NaN,NaN,Occasion,NaN,1970-01-01 00:00:00,1970-01-01 00:00:00,...,Noir,NaN,Noir,Roadster,NaN,NaN,790 cm³,0,NaN,KTM 790 DUKE 2020 PREMIERE MAIN ENTRETIEN A JO...
